In [42]:
import pandas as pd
import numpy as np
from newspaper import Article
import re

from bs4 import BeautifulSoup
import requests

In [54]:
all_news = pd.read_csv("../data/all_news.csv")
test_article = all_news.iloc[15000]
test_article.url

'https://www.espn.com/soccer/soccer-transfers/story/4466009/what-eduardo-camavinga-offers-real-madrid-after-transfer-from-rennes'

In [59]:
article = Article(test_article.url)

In [60]:
article.download()
article.parse()

In [63]:
article.images

{'',
 'https://a.espncdn.com/combiner/i?img=%2Fredesign%2Fassets%2Fimg%2Ficons%2FESPN%2Dicon%2Dsoccer.png&w=80&h=80&scale=crop&cquality=40&location=origin',
 'https://a.espncdn.com/combiner/i?img=/photo/2020/0901/r739343_1296x1296_1-1.jpg&w=130&h=130&scale=crop&location=center',
 'https://a.espncdn.com/combiner/i?img=/photo/2021/0420/soc_36-under-21_1x1.jpg&w=130&h=130&scale=crop&location=center',
 'https://a.espncdn.com/combiner/i?img=/photo/2021/0810/r894037_1296x1296_1-1.jpg&w=130&h=130&scale=crop&location=center',
 'https://a2.espncdn.com/combiner/i?img=%2Fphoto%2F2021%2F0419%2Fr842877_1296x729_16%2D9.jpg'}

In [64]:
article.text

"Julien Laurens says Real Madrid should be excited by the potential signing of Eduardo Camavinga from Rennes. (1:46)\n\nEduardo Camavinga, 18, has completed a move to Real Madrid from Rennes, having been on the radar of Europe's top clubs for a few years.\n\nThe midfielder, who is the youngest player since 1932 to be called up by France, shot to worldwide fame with his performances in Ligue 1 and Madrid moved to land him on Transfer Deadline Day for around €30m, one year before his Rennes contract expires.\n\nNow he is at the Bernabeu, what can the LaLiga side expect from the young midfielder? And just how far can his career develop?\n\n- Stream ESPN FC Daily on ESPN+ (U.S. only)\n\n- Don't have ESPN? Get instant access\n\nCamavinga's background\n\nBorn in a refugee camp in Angola, Camavinga moved to France with his parents and five siblings at the age of two. Eight years later, he was just starting out at Rennes' youth academy when the family home burnt down.\n\nHaving raced through y

In [65]:
article.nlp()
article.summary

"Julien Laurens says Real Madrid should be excited by the potential signing of Eduardo Camavinga from Rennes.\n(1:46)Eduardo Camavinga, 18, has completed a move to Real Madrid from Rennes, having been on the radar of Europe's top clubs for a few years.\nHaving raced through youth levels, Camavinga was handed his first-team debut as a 16-year-old toward the end of the 2018-19 season.\nIt was his performances as a holding midfielder that helped launch Camavinga as a rising star in Ligue 1 and for France.\nIn a more advanced midfield role, with slightly less space to roam, Camavinga has demonstrated tactical discipline and patience."

In [66]:
soup = BeautifulSoup(requests.get(test_article.url).text)
links = [a.get('href') for a in soup.find_all('a', href=True) if a.get('href')[0:5] == "https"]

In [67]:
links

['https://www.espncricinfo.com/',
 'https://www.espn.com/college-sports/basketball/recruiting/',
 'https://www.espn.com/college-sports/football/recruiting/',
 'https://www.espn.com/espnradio/index',
 'https://www.espn.com/watch/',
 'https://www.espn.com/espnplus/?om-navmethod=topnav',
 'https://www.espn.com/soccer/team/_/id/2765/united-states',
 'https://www.espn.com/insider/pickcenter/index?sport=soccer',
 'https://www.vividseats.com/sports/soccer/?wsUser=717&wsVar=soccersubnav',
 'https://www.facebook.com/dialog/share?href=https%3A%2F%2Fwww.espn.com%2Fsoccer%2Fsoccer%2Dtransfers%2Fstory%2F4466009%2Fwhat%2Deduardo%2Dcamavinga%2Doffers%2Dreal%2Dmadrid%2Dafter%2Dtransfer%2Dfrom%2Drennes&app_id=116656161708917',
 'https://www.facebook.com/dialog/share?href=https%3A%2F%2Fwww.espn.com%2Fsoccer%2Fsoccer%2Dtransfers%2Fstory%2F4466009%2Fwhat%2Deduardo%2Dcamavinga%2Doffers%2Dreal%2Dmadrid%2Dafter%2Dtransfer%2Dfrom%2Drennes&app_id=116656161708917',
 'https://pinterest.com/pin/create/button/?url